# فراخوانی کتابخانه ها

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import arabic_reshaper
import bidi.algorithm
def text_fa(x):
    x = arabic_reshaper.reshape(x)
    x = bidi.algorithm.get_display(x)
    return x
font = {'family' : 'B Nazanin', 'size' :15}
plt.rc('font', **font)

===============================================================

#  سوال 1

In [ ]:
df = pd.read_csv(r'C:\Users\Red Pc\Desktop\COURSE\پروژه\iranprovs_mortality_monthly.csv')

## گروه بندی داده ها

In [ ]:
data_frame_grouped = df.groupby(['y','m','prov']).agg({'n':'sum'})
data_frame_grouped = data_frame_grouped.reset_index()

## ایجاد دسترسی به داده ها از طریق زمان

In [ ]:
data_frame_grouped['ym'] = data_frame_grouped['y'] + data_frame_grouped['m']/12 - 1/24
ym_covid = 1398 + 10/12 - 1/24
data_frame_grouped = data_frame_grouped[data_frame_grouped.y >= ym_covid - 5]

## ایجاد دیتا فریم جدید برای اضافه کردن داده های پیش بینی و فوت اضافه

In [ ]:
main_dataframe = pd.DataFrame(columns=data_frame_grouped.columns)
main_dataframe['n_predected'] = None
main_dataframe['n_excess'] = None
main_dataframe['excess_ratio'] = None

## بدست آوردن پیش بینی و آمار فوت اضافه

In [ ]:
for i in data_frame_grouped.prov.unique():
    for j in range(1,13):
        dsmp = data_frame_grouped[np.logical_and(data_frame_grouped.prov == i , data_frame_grouped.m == j)]
        dsmp_modeling = dsmp[dsmp.ym < ym_covid]
        linereg = LinearRegression()
        linereg.fit(dsmp_modeling[['y']],dsmp_modeling[['n']])
        dsmp['n_predected'] = linereg.predict(dsmp[['y']])
        dsmp['n_excess'] = dsmp.n - dsmp.n_predected
        dsmp['excess_ratio'] = dsmp.n_excess / dsmp.n_predected
        # if j == 6:
        #     plt.clf()
        #     plt.figure(figsize=(14, 10))
        #     dsmp = df[np.logical_and(df.prov == i, df.m == 6)]
        #     DSMP = dsmp[dsmp.ym >= ym_covid]
        #     plt.scatter(dsmp.ym, dsmp.n, label=text_fa('مرگ و میر پیش از کرونا'))
        #     plt.scatter(DSMP.ym, DSMP.n, color='darkblue', s=65, label=text_fa('مرگ و میر پس از کرونا'))
        #     plt.plot([DSMP.ym, DSMP.ym], [DSMP.n, DSMP.n_predected], color='darkblue', linestyle='--', alpha=.5)
        #     plt.plot(dsmp.ym, dsmp.n_predected, label=text_fa('پیش بینی آمار فوت'))
        #     plt.axvline(ym_covid, linewidth=2, linestyle="--", color='darkred', alpha=0.75, label=text_fa('زمان کرونا'))
        #     plt.title(text_fa(i))
        #     plt.xlabel(text_fa('سال'))
        #     plt.ylabel(text_fa('مرگ و میر'))
        #     plt.legend(fontsize='small')
        #     plt.show()
        main_dataframe = pd.concat([main_dataframe,dsmp],axis = 0)

## جداسازی آمار کرونا

In [ ]:
ds_covid = main_dataframe.ym
ds_covid = main_dataframe[ds_covid > ym_covid]

## رسم نقشه ی حرارتی

In [ ]:
pivot_data = ds_covid.pivot(index='prov', columns= 'ym', values= 'excess_ratio')
fig, ax = plt.subplots(figsize=(21,15))
# Add title to the Heat map
title = text_fa("نقشه حرارتی شدت و ضعف کرونا در هر استان در هر ماه")
# Set the font size and the distance of the title from the plot
plt.title(title,fontsize=15)
ttl = ax.title
ttl.set_position([0.5,1.05])

## تبدیل تاریخ

In [ ]:
ym = pivot_data.columns
y = np.int64(ym)
m = np.int64(np.round((ym + 1/24 - y)*12))
pivot_data.columns = [text_fa(str(y[i])) + '/' + text_fa(str(m[i])) for i in range(len(ym))]

## تبدیل فونت فارسی

In [ ]:
x =list(pivot_data.index)
for i in range(len(x)):
    x[i] = text_fa(x[i])
pivot_data.index = x

## Heatmap

In [ ]:
sns.heatmap(pivot_data,yticklabels =True, xticklabels = True ,annot=pivot_data,fmt=".1f",ax=ax ,annot_kws={'size': 6,'color':'black'},cmap=sns.cubehelix_palette(start=.5, rot=-.5,light=1.2, as_cmap=True),vmin=-.5, vmax=2.5,center=0,cbar_kws={'label': text_fa('نرخ فوت اضافه')})
ax.set(xlabel="", ylabel="")
plt.show()

# سوال 4

In [ ]:

ds = df.groupby(['y','m','prov','age_group']).agg({'n':'sum'})
ds = ds.reset_index()
ds_prov = ds[(ds['age_group'] == '70-74') | (ds['age_group'] == '75-79')]
ds_prov = ds_prov.groupby(['y','m','prov']).agg({'n':'sum'})
ds_prov = ds_prov.reset_index()

ds_prov['ym'] = ds_prov['y'] + ds_prov['m']/12 - 1/24
ym_covid = 1398 + 10/12 - 1/24
ds_prov = ds_prov[ds_prov.y >= ym_covid - 5]


Ds = pd.DataFrame(columns=ds.columns)
Ds['n_predected'] = None
Ds['n_excess'] = None
Ds['excess_ratio'] = None



for i in ds_prov.prov.unique():
    for j in range(1,13):
        dsmp = ds_prov[np.logical_and(ds_prov.prov == i , ds_prov.m == j)]
        dsmp_modeling = dsmp[dsmp.ym < ym_covid]
        linereg = LinearRegression()
        linereg.fit(dsmp_modeling[['y']],dsmp_modeling[['n']])
        dsmp['n_predected'] = linereg.predict(dsmp[['y']])
        dsmp['n_excess'] = dsmp.n - dsmp.n_predected
        dsmp['excess_ratio'] = dsmp.n_excess / dsmp.n_predected
        Ds = pd.concat([Ds,dsmp],axis = 0)

ds_covid = Ds.ym
ds_covid = Ds[ds_covid > ym_covid]

## میانگین گیری از داده های رده سنی 70 تا 80 سال

In [ ]:
evaluate_prov = ds_covid.groupby('prov')['excess_ratio'].mean()
evaluate_prov = pd.DataFrame(evaluate_prov)
evaluate_prov.reset_index()
mean_excess_ratio= float(evaluate_prov.mean())
x =list(evaluate_prov.index)
for i in range(len(x)):
    x[i] = text_fa(x[i])
evaluate_prov.index = x

## رسم نمودار پراکندگی برای استانها

In [ ]:
evaluate_prov.sort_values('excess_ratio', ascending=True, inplace=True)
# evaluate_prov.to_excel('evaluate_prov.xlsx')
fig, ax =plt.subplots(figsize =(14,18))
title = text_fa("پراکندگی نرخ فوت اضافه برای استانها (سنین مشابه 70 تا 80 سال)")
plt.title(title,fontsize=20)
ttl = ax.title
ttl.set_position([0.5,1.05])
plt.scatter(evaluate_prov.index,evaluate_prov,s = 80,c='darkblue',label =text_fa('میانگین نرخ فوت اضافه هر استان سنین 70 تا 80'),alpha=.8)
plt.xticks(rotation = 90)
plt.axhline(y = mean_excess_ratio,c = 'royalblue', alpha = .8, linestyle = '--',label =text_fa('میانگین نرخ فوت اضافه استانها'))
ax.set(xlabel="", ylabel=text_fa("نرخ فوت اضافه سنین 70 تا 80 سال"))
plt.legend()
plt.show()

## رسم نمودار هیستوگرام و توزیع نرمال

In [ ]:
from scipy.stats import norm
fig, ax =plt.subplots(figsize =(15,15))
title = text_fa("نمودار هیستوگرام نرخ فوت اضافه برای استانها (سنین مشابه 70 تا 80 سال)")
plt.title(title,fontsize=20,fontweight = "bold")
ttl = ax.title
ttl.set_position([0.5,1.05])
mu, std = norm.fit(evaluate_prov)
plt.hist(evaluate_prov,bins=10,density=True,color ='darkblue',alpha = .6)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, '--', linewidth=5,color ='black')
plt.show()

## تست داده ها برای اینکه از توزیع نرمال پیروی می کنند

In [ ]:
from scipy.stats import shapiro
# Perform Shapiro-Wilk test
stat, p = shapiro(evaluate_prov)
print(f'Statistic: {stat}, p-value: {p}')

if p > 0.05:
    print("Data follows a normal distribution")
else:
    print("Data does not follow a normal distribution")

## بررسی استانهایی که از توزیع نرمال پیروی نمی کنند

In [ ]:
from scipy.stats import norm
# Parameters of the normal distribution (mean and standard deviation)
mean = 0
std_dev = 1
# Confidence level for the interval
confidence_level = 0.95
# Calculate the critical values for the confidence interval
lower_critical_value = norm.ppf((1 - confidence_level) / 2, loc=mean, scale=std_dev)
upper_critical_value = norm.ppf(1 - (1 - confidence_level) / 2, loc=mean, scale=std_dev)
# Initialize a list to store rows of data points outside the curve
outside_rows = []
# Loop through each row in the DataFrame
for index, row in evaluate_prov.iterrows():
    data_point = row['excess_ratio']
    if data_point < lower_critical_value or data_point > upper_critical_value:
        outside_rows.append(row)

## Create a new DataFrame from the list of outside rows

In [ ]:
outside_df = pd.DataFrame(outside_rows)

## Print the DataFrame containing data points outside the curve

In [ ]:
print("Data points outside the curve of the distribution:")